In [2]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [33]:
html = WebBaseLoader("https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md")
pages = html.load_and_split()

In [34]:
pages

[Document(page_content='anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nPackages\n        Host and manage packages\n      \n\n\n\n\n\n\n\nSecurity\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev environments\n      \n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nCode review\n        Manage code changes\n      \n\n\n\n\n\n\n\nIssues\n        Plan and track work\n      \n\n\n\n\n\n\n\nDiscussions\n        Collaborate outside of code\n      \n\n\n\n\nExplore\n\n\n\n      All features\n\n    \n\n

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ]

In [37]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=5,
    separators=separators
)

In [38]:
docs = r_splitter.split_documents(pages)

In [39]:
docs

[Document(page_content='anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub', metadata={'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub', 'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en'}),
 Document(page_content='Skip to content\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu', metadata={'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub', 'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en'}),
 Document(page_content='Toggle navigation\n\n\n\n\n\n\n\n\n\n

In [40]:
len(docs)

52

In [41]:
docs[0].page_content

'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'

In [12]:
# Read API KEY from file
import json

file_path = 'llms_vault.json'
llm_key = 'GOOGLE_API_KEY'

try:
    with open(file_path,'r') as f:
        api_key = json.loads(f.read()).get(llm_key)
        if not api_key:
            raise ValueError(f"Some issue with '{llm_key}' from '{file_path}'. Verify the key:'{llm_key}' if it exists!")
except FileNotFoundError:
    raise FileNotFoundError(f"File related issue encountered, verify file path and name '{file_path}'")

# https://ai.google.dev/api
import google.generativeai as genai
genai.configure(api_key=api_key) # genai.configure(api_key=os.environ[llm_key]) 

In [42]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings # models/text-embedding-004	

import os
os.environ["GOOGLE_API_KEY"] = api_key

In [43]:
#result = genai.embed_content(model="models/text-embedding-004", content=docsr)

embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", # text-embedding-004
    google_api_key=api_key,
    )

In [44]:
embedding_model

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022B4CF7A1E0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [45]:
from langchain.vectorstores import Chroma

In [17]:
persist_directory = 'docs\\chromaE'

In [ ]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_directory,
)

vectordb = Chroma.from_documents(
    documents=docsr,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [19]:
vectordb

In [20]:
print(vectordb._collection.count())

50


In [21]:
googleDb = Chroma(
    persist_directory = persist_directory,
    embedding_function=embedding_model
    )
print("vector:",googleDb._collection.count(),"chunks.")

vector: 50 chunks.


In [22]:
googleDb

In [23]:
def search(question='') -> list:
    docs = googleDb.similarity_search(question, k=3)
    answers=[]
    for doc in docs:
        answers.append(doc.page_content[:200])
    
    return answers

In [26]:
search(question="list me some places where can i contribute to")

['Code\n\n\n\n\n\n\n\n\n\n\n          Issues\n\n\n\n\n\n\n\n\n\n\n          Pull requests',
 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub',
 'We read every piece of feedback, and take your input very seriously.']

In [28]:
search(question="list available URLs with text as provided in triple ticks '''github.io'''")

['Search or jump to...\n\n\n\n\n\n\n\nSearch code, repositories, users, issues, pull requests...',
 'Insights\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFooter\n\n\n\n\n\n\n\n\n        © 2024 GitHub,\xa0Inc.',
 'Dismiss alert\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        anishchapagain\n \n/\n\nanishchapagain.github.io\n\nPublic']

In [29]:
search(question="what is there mentioned as Willing to get trained in")

['Notifications\n You must be signed in to change notification settings\n\n\n \n\nFork\n    0',
 'Documentation\n\n    \n\n\n\n\n\n      GitHub Skills\n\n    \n\n\n\n\n\n      Blog',
 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub']

In [30]:
search("what we can work together in any topics")

['Discussions\n        Collaborate outside of code\n      \n\n\n\n\nExplore\n\n\n\n      All features',
 'Issues\n        Plan and track work',
 'Code\n\n\n\n\n\n\n\n\n\n\n          Issues\n\n\n\n\n\n\n\n\n\n\n          Pull requests']

In [32]:
search("list any text that's similar to honesty")

['Search syntax tips\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Provide feedback',
 'Footer navigation\n\n\nTerms\n\n\nPrivacy\n\n\nSecurity\n\n\nStatus\n\n\nDocs\n\n\nContact',
 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub']